## Logistic Regression

Logistic regression is a simple machine learning algorithm for classification. It computes the weighted sum of its inputs and outputs an *activation* that turns the weighted sum into fixed interval (here: [0,1]). This allows us to interpret the logistic regression output as the probability of being of a class or not.
In this toy example we are going to use it to decide, to which of two distributions a data point belongs.

### Setup

For a detailed explanation of the used modules, please refer to the respective sections in the [introductory notebook](0_MNIST_dataset.ipynb).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(842424)

### Generation of Artificial Data

First we generate some artificial data for our toy model to learn on.
The task is to predict to which normal distributions one individual data point belongs.

In [ ]:
def generate_data(n_samples=1000, input_dim=2):
    """
    Return coordinates distributed in 2D space by two different gaussian distributions.
    
    Returns
    -------
    x : np.array[n_samples, input_dim]
        coordinates
    y : np.array[n_samples]
        labels
    """
    half_samples = n_samples // 2
    
    # generate the blobs
    x1 = np.random.normal(1., 0.25, size=(half_samples, input_dim))
    x2 = np.random.normal(2., 0.30, size=(half_samples, input_dim))
    
    # create matching labels
    y1 = np.zeros(half_samples)
    y2 = np.ones(half_samples)
    
    return np.concatenate((x1, x2)), np.concatenate((y1, y2))

data, labels = generate_data()
data.shape, labels.shape

We shuffle the data to improve covergence behaviour and to more closely emulate real data.

In [ ]:
shuffled_indices = np.arange(data.shape[0])
np.random.shuffle(shuffled_indices)

data, labels = data[shuffled_indices], labels[shuffled_indices]

And plot the points colored by label.

In [ ]:
def plot_data(data, labels):
    """
    Plot data colored by labels.
    
    Parameters
    ----------
    data : np.array[n_samples, input_dim]
    labels : np.array[input_dim]
    """
    plt.scatter(data[:, 0], data[:, 1], c=labels, vmin=0.0, vmax=1.0)
    plt.show()
    
plot_data(data, labels)

### The Logistic Function
The logistic (or sigmoid) function gives the predicted probability of the tested hypothesis being true.
In our case: does the input belong to the yellow distribution?

$$\sigma(z) = \frac{1}{1 + \text{e}^{-z}}$$

The activation function of the logistic regression is applied to the dot product of the weights of the model and the input.
To optimize the model with a gradient descent optimizer, the activation function has to be differentiable.

In [ ]:
def sigmoid(z):
    """Return the sigmoid of input z."""
    ##############
    return the sigmoid of z
    ##############

def plot_sigmoid():
    """
    Plot sigmoid function from -10 to 10.
    """
    plot_range = np.arange(-10, 10, 0.01)
    ##############
    plt the sigmoid in the given range
    ##############
    plt.show()
    
plot_sigmoid()

### Logistic Regression Model

The logistic regression is a very simple linear model. It encodes a single straight line (in 2D). Everything beyond the line does not belong to the class.

In the multiclass case, one simply trains multiple models.

In [ ]:
def lr_predict(weights, x):
    """
    Return the prediction of the model for input x with the given weights.
    
    Parameters
    ----------
    weights: the weights of the model to be learned. There is one weight for every input dimension plus a bias.
    x: input data where the 0th input should be 1.0 to take the bias into account in a simple dot product.
    
    Returns:
    ----------
    The activation of the logistic regression, the sigmoid of the dot product of weights and input.
    """
    ##############
    return the prediction for x

    weights = initialize the weights and dont forget the bias
    ##############
    
ones = np.ones((data.shape[0], 1,))
plot_data(data, lr_predict(weights, np.hstack([ones, data])))
plot_data(data, np.around(lr_predict(weights, np.hstack([ones, data]))))

### The Loss Function

The loss function measures how well the model performs. If the model predicts all samples correctly it would be 0.
Here we use Mean Squared Error:

$$ MSE(\textrm{samples}) = \frac{1}{2 |\textrm{samples}|} \sum_{x \in \textrm{samples}} \left(\hat{y}(x) - y_{\textrm{label}}\right)^2$$

To minimize the loss, we also compute the gradient w.r.t. a single weight.

$$ \nabla = (\hat{y}(x) - y_{\textrm{label}}) \cdot y(x)^2 \cdot \textrm{e}^{-w \cdot x} \cdot x$$

where the weight vector is $w$ with the bias $w_0$.

In [ ]:
def lr_loss(weights, x, label):
    """Return the loss and the gradient with respect to the weights.
    
    Parameters
    ----------
    weights : weights of the model to be learned, where weights[0] is the bias
    x : single input sample, x[0] is assumed to be 1
    label : label belonging to x
    
    Returns
    -----------
    loss : mean square error loss for a single sample
    gradient : np.array([weights.shape]) 
        gradient of loss with respect to labels
    """
    loss = 0.
    features = x.shape[0]
    
    ##############
    y = compute the prediction for x
    
    loss = compute the loss for y
    gradient = compute the gradient of the loss
    ##############
    return loss, gradient

lr_loss(weights, np.concatenate([[1], data[0]]), labels[0])

### Training the Model Manually
To train the model, we first initialize the weights (plus bias) and compute the corresponding loss.
To comfortably incorporate the bias, prepend $x_0 = 1$ to the input data so you can just compute the dot product in the functions above.
We then update the weights with the gradient of the loss, until the model converges.
Since loss is not a very intuitable quantity, we also print the accuracy (percentage of correctly classified samples).

In [ ]:
def lr_train(weights, x, labels, epochs=100, lr=0.001):
    """
    Train the model, i.e. update the weights following the negative gradient until the model converges.
    """
    n_samples = x.shape[0]
    ones = np.ones((n_samples, 1,))
    x = np.hstack([ones, x])
    
    for epoch in range(epochs):
        ##############
        compute loss and accuracy and print them during the training
        
        for n in range(n_samples):
            compute loss and gradient
            update weights
            print training status
        ##############
    return weights

weights = lr_train(weights, data, labels)

In [ ]:
ones = np.ones((data.shape[0], 1,))
plot_data(data, lr_predict(weights, np.hstack([ones, data])))
plot_data(data, np.around(lr_predict(weights, np.hstack([ones, data]))))

### Keras
Building a model can often be simplified with a high level API like Keras.
Our simple model in the Keras vocabulary is defined as a $\texttt{Sequential}$ model with a single $\texttt{Dense}$ layer 
with a sigmoid activation function, $\texttt{mse}$ loss and a stochastic gradient descent ($\texttt{SGD}$) optimizer.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

The simplest model in Keras is $\texttt{Sequential}$, a linear stack of layers. The layers can be passed to the constructor as a list, or $\texttt{add}$ed later.

The $\texttt{Dense}$ layer encodes:
$$ f(W \cdot x)$$
where $W \in \mathbb{R}^{\text{m x 1+n}}$, x = (1, $\textrm{input}_1$, .. , $\textrm{input}_{\textrm{n}}$) and f applies the activation function to each element in the resulting vector.
$m$ and $n$ are output- and input dimensions respectively ($m = 1$ in this case).
In short: every output is connected to every input plus a bias.

Keras also always uses minibatches by default. Why?

It is then "compiled" with a stochastic gradient descent optimizer and an mean square error loss.
These parameters can be given by keyword or custom objects. E.g. here the default learning rate of the SGD optimizer is too slow for this simple demonstration so we supply a custom $\texttt{optimizer}$.

In [ ]:
def build_model(data):
    """
    Build a simple perceptron for the given input data.
    """
    model = Sequential()
    model.add(Dense(1, input_dim=data.shape[1], activation='sigmoid'))
    
    model.compile(optimizer=SGD(lr=0.05), loss='mse', metrics=['accuracy'])
    
    return model

model = build_model(data)

The training is done with the $\texttt{fit}$ method.

In [ ]:
def train_model(model, data, labels, epochs=100, batch_size=50):
    """
    Train the given model with the given data and labels for the given epochs.
    """
    model.fit(data, labels, batch_size=batch_size, epochs=epochs)
    
train_model(model, data, labels)

This shows the prediction of the network in a more systematic way.

In [ ]:
def plot_prediction_grid(model, grid_size=30):
    """
    Plot a 2D grid and the network prediction at every grid point for the given grid size.
    """
    x_coord = np.zeros((grid_size, grid_size))
    y_coord = np.zeros((grid_size, grid_size))
    
    for i in range(grid_size):
        for j in range(grid_size):
            x_coord[i, j] = i / 10.0
            y_coord[i, j] = j / 10.0
            
    x_coord = x_coord.reshape(-1, 1)
    y_coord = y_coord.reshape(-1, 1)    
    
    prediction = model.predict(np.hstack([x_coord, y_coord]))
    plt.scatter(x_coord.flatten(), y_coord.flatten(), c=prediction.flatten())
    plt.show()

plot_prediction_grid(model)

In [ ]:
plot_data(data, model.predict(data).flatten())

### Bonus:
Why is a simple logistic regression not the optimal solution for this task?

What is the property required of an actual model solution?